In [1]:
from Mollification import *

import trimesh
import os
import time

root_folder = os.getcwd()

In [2]:
[V,F]= igl.read_triangle_mesh(os.path.join(
            # root_folder, "../../GC/TestEdgeLengths", "11.stl"))
            root_folder, "../../GC/Thingi10K/Thingi10K/raw_meshes", "93842.stl"))


In [9]:
F = np.array([F]).repeat(2, axis=0)
print(F.shape)

(2, 144046, 3)


In [4]:
print(V, F)

[[-0.6301  1.4997 -0.5411]
 [-0.6205  1.5054 -0.5423]
 [-0.621   1.4978 -0.5417]
 ...
 [-0.1342  0.0502  0.5093]
 [-0.1636  0.0409  0.5287]
 [-0.176   0.04    0.5318]] [[    0     1     2]
 [    2     1     3]
 [    2     3     4]
 ...
 [63726 71981 63728]
 [71998 26888 26890]
 [26923 26922 72017]]


In [3]:
delta = 1e-2
FL0 = igl.edge_lengths(V, F)
# time before
end0 = time.time()
FL1 = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
end1 = time.time()
print("Constant Epsilon took: ", end1 - end0)

FL2 = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP)
end2 = time.time()
print("One by one step took: ", end2 - end1)

FL3 = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)
end3 = time.time()
print("One by one interpolated took: ", end3 - end2)

FL4 = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN)
end4 = time.time()
print("Local least mollification Manhattan took: ", end4 - end3)

FL5 = IntrinsicMollification(V, F, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN)
end5 = time.time()
print("Local least mollification Euclidean took: ", end5 - end4)

Constant Epsilon took:  0.05381584167480469
One by one step took:  1.9845805168151855
One by one interpolated took:  1.9355261325836182
Local least mollification Manhattan took:  3.007106065750122
Local least mollification Euclidean took:  6.898361444473267


In [6]:
print(np.sum(FL1 - FL0) / np.sum(FL0))
print(np.sum(FL2 - FL0) / np.sum(FL0))
print(np.sum(FL3 - FL0) / np.sum(FL0))
print(np.sum(FL4 - FL0) / np.sum(FL0))
print(np.sum(FL5 - FL0) / np.sum(FL0))

0.009999999918769322
2.9532941867876297e-06
5.203186307362574e-06
2.800840003085134e-06
2.814440321656371e-06


In [8]:
b = np.array([- delta,   - delta,    - delta,    0,     0,    0])
print(b, b[3])

[-0.1 -0.1 -0.1  0.   0.   0. ] 0.0
